In [3]:
import pandas as pd  
import numpy as np  
import pandas_datareader as pdr
import matplotlib.pyplot as plt  
import datetime 
import yfinance as yf
from nsepy import get_history
from datetime import date
import datetime as dt
from nsetools import Nse
# import seaborn as sns  
from nsetools import Nse 
import datetime 
from datetime import datetime, timedelta
from nsepy.history import get_price_list
import pandas_datareader.data as web
from datetime import datetime, timedelta
import datetime

def all_time_high(shrape_look_back_period,freq_day):
    
   

    prices = get_price_list(dt=date(2022,1,17)) #change datecjkbas
    prices =prices.loc[prices['SERIES'] == 'EQ']
    condition = prices['TOTTRDVAL'] >10_00_00_00_000
#     condition2 = prices['TOTTRDVAL'] >5_00_00_00_000
    prices= prices[condition]
#     prices= prices[condition2]

    df=prices.loc[prices['SERIES'] == 'EQ']
    df= df[['SYMBOL']]
    stock_list=list(df.SYMBOL)

    ticker_list=[]

    for i in range(len(stock_list)):
        end='.NS'
        symbol=stock_list[i] + end
        ticker_list.append(symbol)

    print(ticker_list)
    print(f"number of stocks is {len(ticker_list)}")

    start = datetime.datetime(2007, 1, 1)
    end = datetime.datetime(2022, 1, 17)

    close_price = web.DataReader(ticker_list, 'yahoo', start, end)['Adj Close']
    print(close_price)
    df=close_price
    df=df.replace(np.nan,0)

    print(df)

    ###############################################################################################################################
    #PARAMETERS

    """
    SHARPE RATIO
    """
    df_pct=df.pct_change()
    df_pct_sum=df_pct.rolling(window=shrape_look_back_period).sum()
    df_std=df_pct.rolling(window=shrape_look_back_period).std()
    df_sharpe_ratio=df_pct_sum.div(df_std)
    df_sharpe_ratio=df_sharpe_ratio.replace(np.nan, 0)
    # print(df_sharpe_ratio)

    """
    ALL-TIME-HIGH-ALGO
    """

    long_algo=df.shift(1).rolling(window=shrape_look_back_period).max()
    long_algo=long_algo.replace(np.nan,0)
    long_algo_signal=df > long_algo
    long_algo_signal.columns=df.columns
    # print(df.columns)
    # print(long_algo_signal)

###############################################################################################################################
    #TRADING DAYS
    date_set=df.index
    date_set_df=pd.DataFrame(date_set)
    date_set_df.columns=['date']

    date_set_df['trading_day']=0

    for i in range(len(date_set_df)):
        
        if i%freq_day ==0:
            date_set_df['trading_day'].iloc[i]=date_set_df['date'].iloc[i]
           
        else:
            date_set_df['trading_day'].iloc[i]=0
            

    condition=date_set_df['trading_day'] !=0
    date_set_df=date_set_df[condition]

    # print(trading_day)
##################################################################################################################################
#STOCK_LIST

    stock_list=[]
    trading_day=date_set_df['trading_day']
    trading_day=np.array(trading_day)

    for i in range(len(trading_day)):
        df_new=df_sharpe_ratio.copy()
        df_new=df_new.loc[trading_day[i]]
    
        df_new=pd.DataFrame(df_new)
        df_new.columns=['sharpe_ratio']
        df_new['stock']=df_new.index
        df_new=df_new.reset_index(drop=True)
        # print(df_new)

        df_all_time=long_algo_signal.copy()
        df_all_time=df_all_time.loc[trading_day[i]]
        df_all_time=pd.DataFrame(df_all_time)
        df_all_time.columns=['condition']
        df_all_time['stock']=df_all_time.index
        df_all_time=df_all_time.reset_index(drop=True)
        condition=df_all_time['condition'] !=False
        df_all_time=df_all_time[condition]
        # print(df_all_time)

        inner_join = pd.merge(df_new,  df_all_time,  on ='stock',  how ='inner') 
        inner_join=inner_join.sort_values(by='sharpe_ratio', ascending=False)
        inner_join=inner_join['stock']
        if len(ticker_list) > 20:
            inner_join_stock=inner_join[:20]
        else:
            inner_join_stock=inner_join[:len(ticker_list)]
        inner_join_stock=np.array(inner_join_stock)
        # print(inner_join_stock)
        
        stock_list.append(inner_join_stock)
    
    
    stock_list = pd.DataFrame(stock_list)
    stock_list = stock_list.reset_index(drop=True)
    trading_day = pd.DataFrame(trading_day)
    print(stock_list)
    
    stock_columns = ['date']
    for i in range(1,len(stock_list.columns)+1):
        stock_columns.append(f'stock_{i}')

    buy_list_df=pd.concat([trading_day, stock_list], axis=1)
    buy_list_df.columns= stock_columns
    buy_list_df=buy_list_df.replace(np.nan,0)
    print(buy_list_df)

    # buy_list_df.to_csv(r"C:\Users\DeepakShenoy\Desktop\Quantitative Research\Equity_Strategy_Backtest\buy_list_df_all_time_high.csv")

    ###################################################################################################################################################################

    """
    Calcaulte the entry price of list of stocks selected by system

    """
    #ENTRY_PRICE_LIST

    price_list_buy=[]
    trading_day=date_set_df['trading_day']
    trading_day=np.array(trading_day)

    for i in range(len(trading_day)):

        df_new=df.copy()
        df_new=df_new.loc[trading_day[i]]
        df_new=pd.DataFrame(df_new)
        df_new.columns=['price']
        df_new['stock']=df_new.index
        df_new=df_new.reset_index(drop=True)

        buy_list_new=buy_list_df.copy()
        buy_list_new=buy_list_new.set_index('date')
        buy_list_new=buy_list_new.loc[trading_day[i]]
        buy_list_new=pd.DataFrame(buy_list_new)
        buy_list_new.columns=['stock']
        buy_list_new=buy_list_new.reset_index(drop=True)

        inner_join = pd.merge(df_new,  buy_list_new,  on ='stock',  how ='inner') 
        inner_join_price=inner_join['price']
        price_list_buy.append(inner_join_price)
        

    price_list_buy=pd.DataFrame(price_list_buy)
    price_list_buy=price_list_buy.reset_index(drop=True)
    trading_day=pd.DataFrame(trading_day)
    price_list_df_entry=pd.concat([trading_day, price_list_buy], axis=1)
    price_list_df_entry.columns=stock_columns
    price_list_df_entry=price_list_df_entry.iloc[:-1]
    price_list_df_entry=price_list_df_entry.replace(np.nan,0)

    print(price_list_df_entry)

    ##############################################################################################################################################################################
    """
    Calculate the exit price of stocks as per generated by system
    """
    #EXIT_PRICE_LIST

    price_list_exit=[]
    trading_day=date_set_df['trading_day']
    trading_day=np.array(trading_day)

    for i in range(len(trading_day)-1):
        
        df_new=df.copy()
        df_new=df_new.loc[trading_day[i+1]]
        df_new=pd.DataFrame(df_new)
        df_new.columns=['price']
        df_new['stock']=df_new.index
        df_new=df_new.reset_index(drop=True)

        buy_list_new=buy_list_df.copy()
        buy_list_new=buy_list_new.set_index('date')
        buy_list_new=buy_list_new.loc[trading_day[i]]
        buy_list_new=pd.DataFrame(buy_list_new)
        buy_list_new.columns=['stock']
        buy_list_new=buy_list_new.reset_index(drop=True)

        inner_join = pd.merge(df_new,  buy_list_new,  on ='stock',  how ='inner') 
        inner_join_price=inner_join['price']
        price_list_exit.append(inner_join_price)

    price_list_exit=pd.DataFrame(price_list_exit)
    price_list_exit=price_list_exit.reset_index(drop=True)
    trading_day=pd.DataFrame(trading_day)
    price_list_df_exit=pd.concat([trading_day, price_list_exit], axis=1)
    price_list_df_exit.columns= stock_columns
    price_list_df_exit['date']=price_list_df_exit['date'].shift(-1)
    price_list_df_exit=price_list_df_exit.iloc[:-1]
    price_list_df_exit=price_list_df_exit.replace(np.nan,0)
    print(price_list_df_exit)

      
# ##################################################################################################################################################################################
    """
    Calculate the month on month return generated by system
    """

    entry_price=price_list_df_entry.copy()
    entry_price=entry_price.drop('date', axis='columns')
    # print(entry_price)

    exit_price=price_list_df_exit.copy()
    exit_price=exit_price.drop('date', axis='columns')
    # print(exit_price)
    
    diff_df=exit_price.sub(entry_price)
    diff_pct=diff_df.div(entry_price)
    diff_pct=diff_pct*(100/20)
    diff_pct=diff_pct*0.01

    absolute_return=diff_pct.sum(axis=1)
    absolute_return=pd.DataFrame(absolute_return)
    absolute_return.columns=['absolute_return']
    absolute_return['absolute_return']=absolute_return['absolute_return'].shift(1)
    trading_day=np.array(trading_day)
    trading_day=pd.DataFrame(trading_day)
    absolute_return['date']=trading_day
    absolute_return=absolute_return.replace(np.nan,0)

    print(absolute_return)

    ####################################################################################################################################################################
#PORTFOLIO_ANALYSIS

    absolute_return['portfolio']=0

    for i in range(len(absolute_return)):

        if i==0:
            absolute_return['portfolio'].iloc[i]=100
        else:
            absolute_return['portfolio'].iloc[i]= absolute_return['portfolio'].iloc[i-1]*(1+absolute_return['absolute_return'].iloc[i] )

    
    absolute_return['date']=pd.to_datetime(absolute_return['date'])

    absolute_return['date']=pd.to_datetime(absolute_return['date'])
    
    absolute_return = absolute_return.set_index(absolute_return['date'])
    
    cagr=(((absolute_return['portfolio'].iloc[-1]/(absolute_return['portfolio'].iloc[0]))**(1/13))-1)*100

    # print(absolute_return)
    #plt.plot( absolute_return['portfolio'])
    #plt.title("Equity_Curve")
    #plt.show()

    window = 50
    Roll_Max_portfolio = absolute_return['portfolio'].rolling(window, min_periods=1).max()
    Daily_Drawdown_portfolio = (absolute_return['portfolio']/Roll_Max_portfolio - 1)*100

    max_return=absolute_return['absolute_return'].max()*(100)
    min_return=absolute_return['absolute_return'].min()*(100)
    
    absolute_return['result'] =np.where( absolute_return['absolute_return'] > 0, 1 ,-1)

    positive_condition=absolute_return['result'] ==1
    absolute_return_positive = absolute_return[positive_condition]
    
    neagtive_condition=absolute_return['result'] == -1
    absolute_return_negative = absolute_return[neagtive_condition]
    
    print(f"max_return is {max_return} ")
    print(f"min_return is {min_return}")
    print(f"No of Positive months is { absolute_return_positive['result'].sum(axis=0)}")
    print(f"No of Negative months is { absolute_return_negative['result'].sum(axis=0)*(-1)}")
    print(f"CAGR is {cagr}")
    print(f"Max Drawdown is  {Daily_Drawdown_portfolio.min()}")

    #plt.plot(Daily_Drawdown_portfolio, color="blue", label="PORTFOLIO")
    #plt.title("DRAWDOWN")
    #plt.legend()
    #plt.show()

    print(buy_list_df.iloc[-1])


all_time_high(60,15)
#kdfn all_time_high(shrape_look_back_period,freq_day

['AXISBANK.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'INFY.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TCS.NS']
number of stocks is 7
Symbols     AXISBANK.NS   HCLTECH.NS  HDFCBANK.NS      INFY.NS  TATAMOTORS.NS  \
Date                                                                            
2007-01-02    82.994698    58.525436    97.000496   210.516205     156.477127   
2007-01-03    84.191994    57.875420    96.574425   214.101044     157.950180   
2007-01-04    84.121025    56.807968    95.191933   211.687988     159.440063   
2007-01-05    83.872711    56.465538    95.812935   210.738510     157.807068   
2007-01-08    82.577858    55.031826    92.916504   204.305359     156.308807   
...                 ...          ...          ...          ...            ...   
2022-01-11   743.250000  1332.445557  1565.900024  1855.599976     501.299988   
2022-01-12   746.849976  1340.874634  1556.650024  1877.449951     507.250000   
2022-01-13   740.700012  1323.123901  1528.000000  1896.800049     51

C:\Users\pavan\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


               0           1            2        3              4  \
0    AXISBANK.NS  HCLTECH.NS  HDFCBANK.NS  INFY.NS  TATAMOTORS.NS   
1    AXISBANK.NS  HCLTECH.NS  HDFCBANK.NS  INFY.NS  TATAMOTORS.NS   
2    AXISBANK.NS  HCLTECH.NS  HDFCBANK.NS  INFY.NS  TATAMOTORS.NS   
3    AXISBANK.NS  HCLTECH.NS  HDFCBANK.NS  INFY.NS  TATAMOTORS.NS   
4           None        None         None     None           None   
..           ...         ...          ...      ...            ...   
243  HDFCBANK.NS        None         None     None           None   
244         None        None         None     None           None   
245         None        None         None     None           None   
246         None        None         None     None           None   
247       TCS.NS  HCLTECH.NS         None     None           None   

                5       6  
0    TATASTEEL.NS  TCS.NS  
1    TATASTEEL.NS  TCS.NS  
2    TATASTEEL.NS  TCS.NS  
3    TATASTEEL.NS  TCS.NS  
4            None    None  
.. 

C:\Users\pavan\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [2]:
#10
max_return is 21.66587950262487 
min_return is -9.011658188267768
No of Positive months is 137
No of Negative months is 111
CAGR is 19.991912521297262
Max Drawdown is  -15.034009085534915

SyntaxError: invalid syntax (<ipython-input-2-9214413760b4>, line 4)

In [ ]:
#30
max_return is 16.96865356614333 
min_return is -12.738420509738436
No of Positive months is 119
No of Negative months is 129
CAGR is 9.262287740392082
Max Drawdown is  -12.738420509738436

In [ ]:
#25
max_return is 14.705431680959263 
min_return is -11.893480354605382
No of Positive months is 128
No of Negative months is 120
CAGR is 10.91467528719965
Max Drawdown is  -11.89348035460538